In [1]:
from functools import reduce
import pandas as pd
import numpy as np

## Geographic Relationships

In [2]:
# Take intersected data from QGIS, recreate geoid
geo = pd.read_csv('./populationsim-master/example_wa_16/data/qgis_geo.csv')
geo = geo.drop('Unnamed: 5', axis=1)
geo['REGION'] = '1'
geo['STATE'] = geo['STATE'].astype(str).str.zfill(2)
geo['COUNTY'] = geo['COUNTY'].astype(str).str.zfill(3)
geo['TRACT'] = geo['TRACT'].astype(str).str.zfill(6)
geo['TRACT'] = geo['STATE'] + geo['COUNTY'] + geo['TRACT']
geo.to_csv('./populationsim-master/example_wa_16/data/geo_cross_walk.csv',index=False)

## Seed Samples from PUMS

#### Persons

In [3]:
p_df = pd.read_csv('./data/PUMS_2017_5YR_WA/psam_p53.csv')

In [4]:
pums_data_p = p_df[['SERIALNO','SPORDER','PUMA','PWGTP','PINCP','AGEP','SEX','SCHL','MAR','COW','JWMNP','DPHY','DEYE','DREM','RAC1P']].copy()
pums_data_p.dropna(inplace=True)

# Sample for testing diminishing training data
pums_data_p = pums_data_p.sample(10000)

pums_data_p.to_csv('./populationsim-master/example_wa_16/data/seed_persons.csv', index=False)

#### Households

In [5]:
h_df = pd.read_csv('./data/PUMS_2017_5YR_WA/psam_h53.csv')

In [6]:
pums_data_h = h_df[['SERIALNO','PUMA','WGTP','HINCP','NP','VEH','BLD','ACCESS']].copy()
pums_data_h.dropna(inplace=True)

# Some PUMS households have no people in them; need to be filtered out of dataset
hh_persons = pd.merge(pums_data_h, pums_data_p, how='left', on='SERIALNO')
index = hh_persons['SPORDER'].index[hh_persons['SPORDER'].apply(np.isnan)]
empty_households = hh_persons.iloc[index,:]['SERIALNO'].values
pums_data_h = pums_data_h[~(pums_data_h['SERIALNO'].isin(empty_households))]

# Sample for testing diminishing training data
pums_data_h

pums_data_h.to_csv('./populationsim-master/example_wa_16/data/seed_households.csv', index=False)

## Control Totals from ACS

#### Tract

In [7]:
t2_df = pd.read_csv('./data/ACS_2019_WA/dp02.csv', low_memory=False)
t2_data = t2_df[['GEO_ID',
                'DP02_0060E',
                'DP02_0061E',
                'DP02_0062E',
                'DP02_0063E',
                'DP02_0064E',
                'DP02_0065E',
                'DP02_0066E',
                'DP02_0153E']].copy()
t2_data.columns = ['GEO_ID',
                  'SCHL1',
                  'SCHL2',
                  'SCHL3',
                  'SCHL4',
                  'SCHL5',
                  'SCHL6',
                  'SCHL7',
                  'ACC1']

t3_df = pd.read_csv('./data/ACS_2019_WA/dp03.csv', low_memory=False)
t3_data = t3_df[['GEO_ID',
                'DP03_0051E',
                'DP03_0052E',
                'DP03_0053E',
                'DP03_0054E',
                'DP03_0055E',
                'DP03_0056E',
                'DP03_0057E',
                'DP03_0058E',
                'DP03_0059E',
                'DP03_0060E',
                'DP03_0061E',
                'DP03_0047E',
                'DP03_0048E',
                'DP03_0049E',
                'DP03_0050E',
                'DP03_0003E',
                'DP03_0006E']].copy()
t3_data.columns = ['GEO_ID',
                  'NUM_HH',
                  'HINCP1',
                  'HINCP2',
                  'HINCP3',
                  'HINCP4',
                  'HINCP5',
                  'HINCP6',
                  'HINCP7',
                  'HINCP8',
                  'HINCP9',
                  'HINCP10',
                  'COW1',
                  'COW2',
                  'COW3',
                  'COW4',
                  'ESR1',
                  'ESR2']

t4_df = pd.read_csv('./data/ACS_2019_WA/dp04.csv', low_memory=False)
t4_data = t4_df[['GEO_ID',
                'DP04_0058E',
                'DP04_0059E',
                'DP04_0060E',
                'DP04_0061E',
                'DP04_0007E',
                'DP04_0008E',
                'DP04_0009E',
                'DP04_0010E',
                'DP04_0011E',
                'DP04_0012E',
                'DP04_0013E',
                'DP04_0014E',
                'DP04_0015E']].copy()
t4_data.columns = ['GEO_ID',
                  'VEH1',
                  'VEH2',
                  'VEH3',
                  'VEH4',
                  'BLD1',
                  'BLD2',
                  'BLD3',
                  'BLD4',
                  'BLD5',
                  'BLD6',
                  'BLD7',
                  'BLD8',
                  'BLD9']

t5_df = pd.read_csv('./data/ACS_2019_WA/dp05.csv', low_memory=False)
t5_data = t5_df[['GEO_ID',
                'DP05_0002E',
                'DP05_0003E',
                'DP05_0008E',
                'DP05_0009E',
                'DP05_0010E',
                'DP05_0011E',
                'DP05_0012E',
                'DP05_0013E',
                'DP05_0014E',
                'DP05_0015E',
                'DP05_0016E',
                'DP05_0017E',
                'DP05_0037E',
                'DP05_0038E',
                'DP05_0039E',
                'DP05_0044E',
                'DP05_0052E',
                'DP05_0057E',
                'DP05_0058E']].copy()
t5_data.columns = ['GEO_ID',
                  'SEX1',
                  'SEX2',
                  'AGEP1',  # Start at 20yrs old; there must be people in the seed to meet the controls
                  'AGEP2',
                  'AGEP3',
                  'AGEP4',
                  'AGEP5',
                  'AGEP6',
                  'AGEP7',
                  'AGEP8',
                  'AGEP9',
                  'AGEP10',
                  'RAC1',
                  'RAC2',
                  'RAC3',
                  'RAC4',
                  'RAC5',
                  'RAC6',
                  'RAC7']

t7_df = pd.read_csv('./data/ACS_2019_WA/s2501.csv', low_memory=False)
t7_data = t7_df[['GEO_ID',
                'S2501_C01_002E',
                'S2501_C01_003E',
                'S2501_C01_004E',
                'S2501_C01_005E']].copy()
t7_data.columns = ['GEO_ID',
                  'NP1',
                  'NP2',
                  'NP3',
                  'NP4']

t8_df = pd.read_csv('./data/ACS_2019_WA/s0801.csv', low_memory=False)
t8_data = t8_df[['GEO_ID',
                'S0801_C02_037E',
                'S0801_C02_038E',
                'S0801_C02_039E',
                'S0801_C02_040E',
                'S0801_C02_041E',
                'S0801_C02_042E',
                'S0801_C02_043E',
                'S0801_C02_044E',
                'S0801_C02_045E']].copy()
t8_data.columns = ['GEO_ID',
                  'JWM1',
                  'JWM2',
                  'JWM3',
                  'JWM4',
                  'JWM5',
                  'JWM6',
                  'JWM7',
                  'JWM8',
                  'JWM9']

t9_df = pd.read_csv('./data/ACS_2019_WA/s1810.csv', low_memory=False)
t9_data = t9_df[['GEO_ID',
                'S1810_C02_047E',
                'S1810_C02_029E',
                'S1810_C02_039E',
                'S1810_C02_063E']].copy()
t9_data.columns = ['GEO_ID',
                  'PHY1',
                  'EYE1',
                  'REM1',
                  'OUT1']

In [8]:
# Join data from different census tables
tract_dataframes = [t2_data, t3_data, t4_data, t5_data, t7_data, t8_data, t9_data]
tract_data = reduce(lambda left,right: pd.merge(left,right,on='GEO_ID'), tract_dataframes)
tract_data.dropna(inplace=True)

# Get tract id from the extended geo id, assign region 1
tract_data['TRACT'] = tract_data['GEO_ID'].str.slice(-11,)
tract_data = tract_data.iloc[1:,:]
tract_data.to_csv('./populationsim-master/example_wa_16/data/control_totals_tract.csv', index=False)
tract_data

,GEO_ID,SCHL1,SCHL2,SCHL3,SCHL4,SCHL5,SCHL6,SCHL7,ACC1,NUM_HH,...,JWM5,JWM6,JWM7,JWM8,JWM9,PHY1,EYE1,REM1,OUT1,TRACT
1,1400000US53001950100,53,121,592,409,190,255,155,839,1050,...,3.2,6.9,1.8,7.3,5.3,190,52,114,122,53001950100
2,1400000US53001950200,20,102,343,303,116,171,53,475,615,...,8.3,8.0,10.8,5.0,11.0,144,31,134,74,53001950200
3,1400000US53001950300,916,381,972,416,148,214,183,1218,1655,...,2.5,8.5,4.2,6.6,9.7,200,123,379,167,53001950300
4,1400000US53001950400,475,94,417,318,57,186,116,704,974,...,2.3,20.7,10.3,1.0,2.3,153,69,129,94,53001950400
5,1400000US53001950500,1035,344,750,452,157,97,102,1232,1679,...,2.9,16.2,6.0,2.2,5.4,356,193,172,140,53001950500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1400000US53077940002,485,373,757,664,163,165,128,1034,1316,...,4.5,8.0,6.7,1.5,2.9,393,80,203,197,53077940002
1455,1400000US53077940003,235,219,760,474,195,117,106,745,1017,...,2.5,17.0,3.7,3.5,4.0,171,65,141,136,53077940003
1456,1400000US53077940004,925,579,1074,431,79,102,54,1277,1615,...,5.7,13.2,1.4,2.2,3.4,260,100,213,174,53077940004
1457,1400000US53077940005,721,420,664,555,120,152,75,1084,1408,...,10.3,13.2,2.9,4.4,4.1,265,120,164,160,53077940005


#### Region

In [9]:
# Sum the values across all census tracts to get region marginals
region_data = tract_data.iloc[:,1:-1].apply(pd.to_numeric, errors='coerce')
region_data = pd.DataFrame(region_data.sum()).transpose()
region_data['STATE'] = '53'
region_data['num_hh'] = 2848396
region_data.dropna(inplace=True)
region_data.to_csv('./populationsim-master/example_wa_16/data/control_totals_state.csv', index=False)
region_data

,SCHL1,SCHL2,SCHL3,SCHL4,SCHL5,SCHL6,SCHL7,ACC1,NUM_HH,HINCP1,...,JWM6,JWM7,JWM8,JWM9,PHY1,EYE1,REM1,OUT1,STATE,num_hh
0,186275.0,256174.0,1122330.0,1189880.0,509353.0,1144545.0,693067.0,2516185.0,2848396.0,136214.0,...,19708.4,11284.1,13538.9,17351.3,445243.0,155474.0,361975.0,310612.0,53,2848396
